In [32]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep
import random
import datetime
from tqdm import tqdm
import re
import random as rand

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [33]:
def obtener_tabla(texto_tabla, encabezados, localidad):
    """
    Convierte un texto de una tabla en un DataFrame organizado.

    Args:
        texto_tabla (str): Texto crudo que contiene la tabla, separada por saltos de línea.
        encabezados (list): Lista con los encabezados de columna para el DataFrame.
        localidad (str): El nombre de la localidad o municipio al que corresponde la tabla.

    Returns:
        pd.DataFrame: Un DataFrame que contiene los datos organizados con las columnas correctas,
                      incluyendo la localidad y el mes al que corresponde la tabla.
    """
    
    lista_tabla = texto_tabla.split("\n")
    indice_max_avg_min = lista_tabla.index('Max Avg Min')
    lugar_de_division = indice_max_avg_min - 1
    tabla_final = []
    for i in range(1, lugar_de_division+1):
        tabla_final.append(lista_tabla[i::lugar_de_division])

    df_ini = pd.DataFrame(tabla_final)
    df_final = pd.DataFrame()
    for j in range(df_ini.shape[1]):
        df_inter = df_ini[j].str.split(" ", expand=True)
        df_final = pd.concat([df_final, df_inter], axis=1)

    df_final.columns = df_final.iloc[0]

    mes = df_final.iloc[0,0]
    nuevas_columnas = [encabezados[0]]

    for titulo in encabezados[1:-1]:
        for i in range(1,4):
            patron_regex = r"\(.*?\)"  # Buscamos cualquier texto que esté entre paréntesis, incluyendo los paréntesis.
            nuevas_columnas.append(re.sub(patron_regex, " ", titulo) + df_final.columns[i])

    nuevas_columnas.append("Precipitaciones")

    df_final.columns = nuevas_columnas

    df_final.drop(index=0, inplace=True)
    df_final.reset_index(drop=True, inplace=True)
    df_final.insert(column="Mes", loc=0, value=mes)
    df_final.insert(column="Municipio", loc=0, value=localidad)
    
    return df_final


In [34]:
df = pd.read_csv("datos/coordenadas_municipios.csv")

municipios = df["Municipio"].unique()
lista_municipios_aleatorios = rand.choices(municipios, k=5)


In [35]:
def esperar_y_hacer_click(drivers, ruta_xpath):
    """
    Espera hasta que un elemento esté presente en la página y realiza un clic en él.

    Args:
        drivers (WebDriver): El objeto del navegador (WebDriver) que está siendo utilizado para controlar la página.
        ruta_xpath (str): El XPath del elemento que se está buscando en la página.

    Returns:
        WebElement: El botón o elemento sobre el cual se hizo clic. Retorna `None` si el elemento no se encuentra después de 5 intentos.
    """
    
    boton = None
    intentos = 0
    while True and intentos <= 5:
        try:
            boton = WebDriverWait(drivers, 10).until(EC.presence_of_element_located(("xpath", ruta_xpath)))
            sleep(2)
            boton.click()
            break
        except Exception as e:
            intentos += 1
            print(f"Intento {intentos}: No se ha podido encontrar el botón. Error: {e}")
            continue
    
    return boton


In [36]:
df_concatenado = pd.DataFrame()

for municipio in lista_municipios_aleatorios:
    
    driver = webdriver.Chrome()
    url_wunder = "https://www.wunderground.com/history"
    driver.maximize_window()
    driver.get(url_wunder)
    busqueda = municipio 
    iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("css selector", "#sp_message_iframe_1165301")))
    driver.switch_to.frame(iframe)
    driver.implicitly_wait(20)
    driver.find_element("css selector", "#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button").click()

    sleep(1)

    driver.switch_to.default_content()

    cuadro_texto = driver.find_element('xpath', '//*[@id="historySearch"]')
    cuadro_texto.send_keys(busqueda)

    WebDriverWait(driver, 10).until(EC.presence_of_element_located(("css selector", '#historyForm > search-autocomplete > ul')))
    sleep(1)
    cuadro_texto.send_keys(Keys.ENTER)
    esperar_y_hacer_click(driver, '//*[@id="dateSubmit"]')
    sleep(2)
    esperar_y_hacer_click(driver, '//*[@id="inner-content"]/div[2]/div[1]/div[1]/div[1]/div/lib-link-selector/div/div/div/a[3]')
    sleep(2)
    df_mes = pd.DataFrame()
    for indice_mes in range(9, -1, -1):
        tabla = WebDriverWait(driver, 10).until(EC.presence_of_element_located(('xpath', '//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table')))
        tabla_html = tabla.get_attribute("innerHTML")
        texto_tabla = tabla.text

        sopa_tabla = BeautifulSoup(tabla_html)

        encabezados = []
        for titulo in sopa_tabla.find("tr").findAll("td"):
            encabezados.append(titulo.text)
        
        df_municipio = obtener_tabla(texto_tabla, encabezados, municipio)
        df_mes = pd.concat([df_mes, df_municipio])
        if indice_mes == 0:
            break
        esperar_y_hacer_click(driver, '//*[@id="monthSelection"]')
        esperar_y_hacer_click(driver, f'//*[@id="monthSelection"]/option[{indice_mes}]')
        esperar_y_hacer_click(driver, '//*[@id="dateSubmit"]')

    driver.quit()
    df_concatenado = pd.concat([df_concatenado, df_mes])

In [37]:

df_concatenado

,Municipio,Mes,Time,Temperature Max,Temperature Avg,Temperature Min,Dew Point Max,Dew Point Avg,Dew Point Min,Humidity Max,Humidity Avg,Humidity Min,Wind Speed Max,Wind Speed Avg,Wind Speed Min,Pressure Max,Pressure Avg,Pressure Min,Precipitaciones
0,talamanca-de-jarama,Oct,1,84,66.8,50,55,45.2,41,72,48.8,23,10,3.6,0,28.0,27.9,27.9,0.00
1,talamanca-de-jarama,Oct,2,79,71.2,63,63,57.0,52,78,62.0,47,18,8.8,0,27.9,27.8,27.7,0.00
2,talamanca-de-jarama,Oct,3,77,67.6,61,59,49.6,37,88,55.5,24,16,5.6,0,27.9,27.8,27.7,0.00
3,talamanca-de-jarama,Oct,4,77,63.9,50,50,46.1,43,82,53.9,36,8,2.3,0,27.9,27.9,27.8,0.00
4,talamanca-de-jarama,Oct,5,81,65.5,50,59,51.0,45,87,61.8,39,17,5.9,0,27.9,27.9,27.8,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,soto-del-real,Jan,27,68,49.5,37,50,42.6,36,93,78.4,49,3,2.1,0,28.3,28.3,28.2,0.00
27,soto-del-real,Jan,28,64,48.1,36,46,41.7,36,100,80.3,48,6,2.5,0,28.3,28.2,28.2,0.00
28,soto-del-real,Jan,29,61,48.7,39,45,41.9,37,93,78.9,48,7,2.8,0,28.3,28.3,28.2,0.00
29,soto-del-real,Jan,30,57,45.6,37,46,41.5,36,100,85.9,63,6,1.9,0,28.4,28.3,28.3,0.00
